In [1]:
import numpy as np
import pandas as pd
import torch
from torch.nn import CrossEntropyLoss
import matplotlib.pyplot as plt
cmap = plt.get_cmap("viridis")

import config as u_config
from graph_models import FullGraphModel

from data_processing import CompleteModelsDataProcessor
from manifold_funcs import manifold_test, extract_details
from manifold_plots import create_dash_app_two_sliders


device_type = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_type)
dtype = torch.float32
batch_size = u_config.batch_size

In [2]:
data_processor = CompleteModelsDataProcessor(u_config)
model = FullGraphModel(data_processor, u_config).to(device)
criterion = CrossEntropyLoss()

checkpoint = torch.load(
    "models/m_2024-06-17 12:58_lrypvwbz.pth",
    # "models/m_2024-07-29 17:05_ietvbnta.pth",
    map_location=device_type,
)
model.load_state_dict(checkpoint["model"])
torch.set_grad_enabled(False)
model.eval()

FullGraphModel(
  (connectome): Connectome()
  (final_fc): Linear(in_features=1, out_features=2, bias=True)
  (decision_making_dropout): Dropout(p=0, inplace=False)
)

In [3]:
test_results, final_plots, accuracy, intermediate, labels = manifold_test(
    model, data_processor, criterion, device, u_config
)

100%|██████████| 375/375 [17:45<00:00,  2.84s/it]

Finished testing with loss 70272.11713060085 and accuracy 0.7659166666666667.


In [4]:
intermediate = intermediate.cpu().numpy()

test_results["radius"], test_results["distance"], test_results["angle"] = zip(
    *test_results["Image"].apply(extract_details)
)
test_results["shape"] = test_results["Image"].str.split("/").str.get(-2)
shape_colors = {"circle": "blue", "star": "red"}
test_results["colour"] = test_results["shape"].map(shape_colors)
bin_num = 30
test_results["distance"] = pd.cut(test_results["distance"], bins=bin_num, labels=False)
test_results["angle"] = pd.cut(test_results["angle"], bins=bin_num, labels=False)

# test_results = pd.read_csv("results/manifolds.csv")

In [5]:
from sklearn.manifold import TSNE

# Perform t-SNE to reduce to 2D for visualization
tsne = TSNE(n_components=3, random_state=42)
reduced_data = tsne.fit_transform(intermediate)
test_results["tsne_Component_1"] = reduced_data[:, 0]
test_results["tsne_Component_2"] = reduced_data[:, 1]
test_results["tsne_Component_3"] = reduced_data[:, 2]

In [6]:
create_dash_app_two_sliders(test_results, "tsne")

In [6]:
from manifold_plots import plot_2D

plot_2D(test_results, "tsne")

/home/eudald/Desktop/doctorat/connectome/venv/lib/python3.10/site-packages/plotly/express/_core.py:1992: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


In [ ]:
import umap

# Perform UMAP to reduce to 2D for visualization
reducer = umap.UMAP(n_components=3)
reduced_data = reducer.fit_transform(intermediate)

test_results["umap_Component_1"] = reduced_data[:, 0]
test_results["umap_Component_2"] = reduced_data[:, 1]
test_results["umap_Component_3"] = reduced_data[:, 2]

In [ ]:
create_dash_app_two_sliders(test_results, "umap")

In [ ]:
from sklearn.decomposition import PCA

pca_2d = PCA(n_components=3)
pca_2d_result = pca_2d.fit_transform(intermediate)

test_results["pca_Component_1"] = pca_2d_result[:, 0]
test_results["pca_Component_2"] = pca_2d_result[:, 1]
test_results["pca_Component_3"] = pca_2d_result[:, 2]

In [ ]:
create_dash_app_two_sliders(test_results, "pca")

# 2D

In [ ]:
%matplotlib inline

from sklearn.manifold import TSNE

# Assuming `all_intermediate_outputs` is the tensor with shape (n, 916)
# and `labels` is a vector with the same length n, containing the labels (circle or star)

# Convert tensors to numpy arrays if they are not already
intermediate = intermediate.cpu().numpy()
# convert labels to numpy array
labels = np.array(labels).reshape(-1)
labels = np.array(["circle" if label == 0 else "star" for label in labels])

# Perform t-SNE to reduce to 2D for visualization
tsne = TSNE(n_components=2, random_state=42)
reduced_data = tsne.fit_transform(intermediate)

# Plotting
plt.figure(figsize=(10, 6))
for label in np.unique(labels):
    indices = labels == label
    plt.scatter(
        reduced_data[indices, 0], 
        reduced_data[indices, 1], 
        label=str(label),
        alpha=0.5
        )

plt.legend()
plt.title("t-SNE of Intermediate Outputs")
plt.xlabel("t-SNE Component 1")
plt.ylabel("t-SNE Component 2")
plt.show()

In [ ]:
from sklearn.manifold import Isomap
np.random.seed(42)
# Perform Isomap to reduce to 2D for visualization
isomap = Isomap(n_components=2)
manifold_data = isomap.fit_transform(intermediate)

# Plotting
plt.figure(figsize=(10, 6))
for label in np.unique(labels):
    indices = labels == label
    plt.scatter(manifold_data[indices, 0], manifold_data[indices, 1], label=str(label))
plt.legend()
plt.title("Isomap of Intermediate Outputs")
plt.xlabel("Isomap Component 1")
plt.ylabel("Isomap Component 2")
plt.show()

In [ ]:
import umap

# Perform UMAP to reduce to 2D for visualization
reducer = umap.UMAP(n_components=2, random_state=42)
reduced_data = reducer.fit_transform(intermediate)

# Plotting
plt.figure(figsize=(10, 6))
for label in np.unique(labels):
    indices = labels == label
    plt.scatter(
        reduced_data[indices, 0], 
        reduced_data[indices, 1], 
        label=str(label),
        alpha=0.5)

plt.legend()
plt.title("UMAP of Intermediate Outputs")
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.show()

In [ ]:
from sklearn.decomposition import PCA

pca_2d = PCA(n_components=2)
pca_2d_result = pca_2d.fit_transform(intermediate)

# Plotting
plt.figure(figsize=(10, 6))
for label in np.unique(labels):
    indices = labels == label
    plt.scatter(
        pca_2d_result[indices, 0],
        pca_2d_result[indices, 1],
        label=str(label),
        alpha=0.5,
    )

plt.legend()
plt.title("PCA of Intermediate Outputs")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.show()

In [ ]:
pca_2d_result

In [ ]:
import plotly.express as px
import pandas as pd

# Perform t-SNE to reduce to 3D for visualization
tsne = TSNE(n_components=3, random_state=42)
tsne_3d = tsne.fit_transform(intermediate)

# Perform UMAP to reduce to 3D for visualization
reducer = umap.UMAP(n_components=3, random_state=42)
umap_3d = reducer.fit_transform(intermediate)

tsne_df = pd.DataFrame(tsne_3d, columns=["Component 1", "Component 2", "Component 3"])
tsne_df["Label"] = labels

umap_df = pd.DataFrame(umap_3d, columns=["Component 1", "Component 2", "Component 3"])
umap_df["Label"] = labels

# Plot t-SNE in 3D using plotly
fig_tsne = px.scatter_3d(
    tsne_df,
    x="Component 1",
    y="Component 2",
    z="Component 3",
    color="Label",
    title="t-SNE 3D of Intermediate Outputs",
)
fig_tsne.show()

In [ ]:

# Plot UMAP in 3D using plotly
fig_umap = px.scatter_3d(
    umap_df,
    x="Component 1",
    y="Component 2",
    z="Component 3",
    color="Label",
    title="UMAP 3D of Intermediate Outputs",
)
fig_umap.show()

In [ ]:
pca = PCA(n_components=3)
pca_3d = pca.fit_transform(intermediate)
pca_df = pd.DataFrame(pca_3d, columns=["Component 1", "Component 2", "Component 3"])
pca_df["Label"] = labels

fig_pca = px.scatter_3d(
    pca_df,
    x="Component 1",
    y="Component 2",
    z="Component 3",
    color="Label",
    title="PCA 3D of Intermediate Outputs",
)
fig_pca.show()